Week 2: Generative Model (VAE) — Train & Validate

Load Week 1 artifacts, train a simple VAE on 20-trading-day forward return windows across 10 stocks, validate realism (KS tests, moments, autocorr/covariance), and save 5,000 synthetic 4-week scenarios for Week 3 (clustering & regime states).

In [2]:
# --- Cell 1: Imports & config ---
import os
import json
import math
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.stats import ks_2samp

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# -------- Paths (Week1 files are in Week2 folder) --------
WEEK2_DIR = Path(__file__).resolve().parent if "__file__" in globals() else Path(".").resolve()
WEEK1_DIR = WEEK2_DIR                               # since you put Week 1 outputs here
OUT_DIR   = WEEK2_DIR                               # save Week 2 outputs here
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Week 1 artifacts
FEAT_FP = WEEK1_DIR / "features_week1.parquet"
RET_FP  = WEEK1_DIR / "returns_week1.parquet"

for f in [FEAT_FP, RET_FP]:
    if not f.exists():
        raise FileNotFoundError(f"Missing file: {f}")

print("Loading Week1 from:", WEEK1_DIR)
print("Saving Week2 outputs to:", OUT_DIR)

# -------- Universe / horizon / training hyperparams --------
TICKERS     = ["AAPL","MSFT","AMZN","GOOGL","META","NVDA","JPM","JNJ","XOM","WMT"]
H           = 20      # horizon: 20 trading days (~4 weeks)
BATCH_SIZE  = 256
EPOCHS      = 60
LR          = 1e-3
LATENT_DIM  = 16
HIDDEN_DIM  = 128
SEED        = 42
N_SAMPLES   = 5000    # scenarios to generate for Week 3

# Reproducibility
torch.manual_seed(SEED)
np.random.seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device



Loading Week1 from: C:\Users\Krish\OneDrive - Curtin\Desktop\University\University\2025 Sem 2\MATH3004\22155584_Week_2
Saving Week2 outputs to: C:\Users\Krish\OneDrive - Curtin\Desktop\University\University\2025 Sem 2\MATH3004\22155584_Week_2


device(type='cpu')

Load Week 1 returns & build forward 20-day windows

In [3]:
# --- Cell 2: Load Week 1 returns and build forward windows ---
returns = pd.read_parquet(RET_FP)

# returns has MultiIndex columns: (ticker, field='ret')
cols = [(t, "ret") for t in TICKERS]
returns = returns.loc[:, cols].copy()
returns.columns = pd.MultiIndex.from_product([TICKERS, ["ret"]], names=["ticker","field"])

R = returns.droplevel("field", axis=1).dropna(how="any")  # [T, N_assets]

# Build strictly forward windows: for each anchor t, sample is returns[t+1 : t+H]
X = []
anchors = []
T_len = len(R)
for i in range(T_len - H):
    win = R.iloc[i+1:i+1+H].to_numpy()
    if np.isfinite(win).all():
        X.append(win)
        anchors.append(R.index[i])

X = np.asarray(X)  # [num_samples, H, N_assets]
print("Training windows:", X.shape, "| anchors:", len(anchors))


Training windows: (2434, 20, 10) | anchors: 2434


Train/val split & scaling (fit on train only)

In [4]:
# --- Cell 3: Split + scale ---
num_samples, H_win, N_assets = X.shape
D = H_win * N_assets

X_flat = X.reshape(num_samples, D)

X_train, X_val = train_test_split(X_flat, test_size=0.2, random_state=SEED, shuffle=True)

scaler = StandardScaler()
X_train_z = scaler.fit_transform(X_train)
X_val_z   = scaler.transform(X_val)

print("Train/Val (z-scored):", X_train_z.shape, X_val_z.shape)

# Save scaler meta
scaler_meta = {
    "mean": scaler.mean_.tolist(),
    "scale": scaler.scale_.tolist(),
    "H": H, "N_assets": N_assets, "tickers": TICKERS
}
with open(OUT_DIR/"vae_scaler.json","w") as f:
    json.dump(scaler_meta, f)


Train/Val (z-scored): (1947, 200) (487, 200)


Dataset/DataLoader

In [5]:
# --- Cell 4: Dataset / DataLoader ---
class WindowDataset(Dataset):
    def __init__(self, Xz):
        self.Xz = torch.tensor(Xz, dtype=torch.float32)
    def __len__(self):
        return self.Xz.shape[0]
    def __getitem__(self, idx):
        return self.Xz[idx]

train_ds = WindowDataset(X_train_z)
val_ds   = WindowDataset(X_val_z)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)


VAE model

In [6]:
# --- Cell 5: VAE model ---
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
        )
        self.mu = nn.Linear(hidden_dim, latent_dim)
        self.logvar = nn.Linear(hidden_dim, latent_dim)

        self.dec = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
        )

    def encode(self, x):
        h = self.enc(x)
        return self.mu(h), self.logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.dec(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon = self.decode(z)
        return recon, mu, logvar

def vae_loss(recon, x, mu, logvar):
    recon_loss = nn.functional.mse_loss(recon, x, reduction='mean')
    kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl, recon_loss, kl

vae = VAE(input_dim=D, hidden_dim=HIDDEN_DIM, latent_dim=LATENT_DIM).to(device)
opt = torch.optim.Adam(vae.parameters(), lr=LR)


Train with early stopping

In [7]:
# --- Cell 6: Train ---
best_val = float("inf")
patience, patience_left = 8, 8

for epoch in range(1, EPOCHS+1):
    vae.train()
    tr_loss = 0.0
    for xb in train_loader:
        xb = xb.to(device)
        recon, mu, logvar = vae(xb)
        loss, rl, kl = vae_loss(recon, xb, mu, logvar)
        opt.zero_grad()
        loss.backward()
        opt.step()
        tr_loss += loss.item() * xb.size(0)
    tr_loss /= len(train_ds)

    vae.eval()
    with torch.no_grad():
        val_loss = 0.0
        for xb in val_loader:
            xb = xb.to(device)
            recon, mu, logvar = vae(xb)
            loss, rl, kl = vae_loss(recon, xb, mu, logvar)
            val_loss += loss.item() * xb.size(0)
        val_loss /= len(val_ds)

    print(f"Epoch {epoch:03d} | train {tr_loss:.4f} | val {val_loss:.4f}")

    if val_loss + 1e-6 < best_val:
        best_val = val_loss
        patience_left = patience
        torch.save(vae.state_dict(), OUT_DIR / "vae_model.pt")
    else:
        patience_left -= 1
        if patience_left == 0:
            print("Early stopping.")
            break

vae.load_state_dict(torch.load(OUT_DIR / "vae_model.pt", map_location=device))
vae.eval()
print("Best val:", best_val)


Epoch 001 | train 1.0119 | val 1.0171
Epoch 002 | train 1.0041 | val 1.0143
Epoch 003 | train 1.0025 | val 1.0134
Epoch 004 | train 1.0019 | val 1.0128
Epoch 005 | train 1.0014 | val 1.0124
Epoch 006 | train 1.0013 | val 1.0124
Epoch 007 | train 1.0009 | val 1.0121
Epoch 008 | train 1.0010 | val 1.0123
Epoch 009 | train 1.0007 | val 1.0122
Epoch 010 | train 1.0007 | val 1.0120
Epoch 011 | train 1.0006 | val 1.0121
Epoch 012 | train 1.0006 | val 1.0119
Epoch 013 | train 1.0005 | val 1.0121
Epoch 014 | train 1.0004 | val 1.0118
Epoch 015 | train 1.0004 | val 1.0118
Epoch 016 | train 1.0004 | val 1.0118
Epoch 017 | train 1.0003 | val 1.0117
Epoch 018 | train 1.0003 | val 1.0114
Epoch 019 | train 1.0003 | val 1.0115
Epoch 020 | train 1.0003 | val 1.0117
Epoch 021 | train 1.0003 | val 1.0115
Epoch 022 | train 1.0002 | val 1.0116
Epoch 023 | train 1.0003 | val 1.0115
Epoch 024 | train 1.0003 | val 1.0117
Epoch 025 | train 1.0002 | val 1.0115
Epoch 026 | train 1.0002 | val 1.0116
Early stoppi

Generate 5,000 synthetic 20-day scenarios & save

In [8]:
# --- Cell 7: Generate synthetic scenarios ---
with torch.no_grad():
    z = torch.randn(N_SAMPLES, LATENT_DIM, device=device)
    gen_z = vae.decode(z).cpu().numpy()  # [N_SAMPLES, D]

# Inverse scale to original return space
gen_flat = scaler.inverse_transform(gen_z)                 # [N_SAMPLES, D]
gen_seq  = gen_flat.reshape(N_SAMPLES, H, len(TICKERS))    # [N, H, N_assets]

# Tidy DataFrame (index = scenario_id, day; columns = tickers)
mux = pd.MultiIndex.from_product([range(N_SAMPLES), range(1, H+1)],
                                 names=["scenario_id","day"])
gen_df = pd.DataFrame(gen_seq.reshape(N_SAMPLES*H, len(TICKERS)),
                      index=mux, columns=TICKERS)

# Save artifacts for Week 3
gen_df.to_parquet(OUT_DIR / "synthetic_paths_20d.parquet")
np.save(OUT_DIR / "synthetic_paths_20d.npy", gen_seq)
print("Saved:", (OUT_DIR / "synthetic_paths_20d.parquet").name)


Saved: synthetic_paths_20d.parquet


Validation: KS tests, moments, autocorr, covariance

In [9]:
# --- Cell 8: Validation metrics ---
report = {}

# Flatten across time for marginal checks
real_all = X.reshape(X.shape[0]*H, len(TICKERS))
gen_all  = gen_seq.reshape(gen_seq.shape[0]*H, len(TICKERS))

# 8.1 KS per asset
ks_results = {}
for i, t in enumerate(TICKERS):
    ks = ks_2samp(real_all[:, i], gen_all[:, i])
    ks_results[t] = {"statistic": float(ks.statistic), "pvalue": float(ks.pvalue)}
report["ks_per_asset"] = ks_results

# 8.2 Moments
moments = {}
for i, t in enumerate(TICKERS):
    moments[t] = {
        "real_mean": float(real_all[:, i].mean()),
        "gen_mean":  float(gen_all[:, i].mean()),
        "real_std":  float(real_all[:, i].std()),
        "gen_std":   float(gen_all[:, i].std()),
    }
report["moments"] = moments

# 8.3 Mean lag-1 autocorr across sequences (per asset)
def mean_lag1_autocorr(seq):  # seq: [num_samples, H, N]
    ac = []
    for j in range(seq.shape[2]):
        x = seq[:, :, j]
        x0 = x[:, :-1] - x[:, :-1].mean(axis=1, keepdims=True)
        x1 = x[:,  1:] - x[:, :-1].mean(axis=1, keepdims=True)
        num = (x1 * x0).mean()
        den = (x0**2).mean()
        ac.append(float(num/den) if den != 0 else 0.0)
    return ac

ac_real = mean_lag1_autocorr(X)
ac_gen  = mean_lag1_autocorr(gen_seq)
report["lag1_autocorr"] = {t: {"real": ac_real[i], "gen": ac_gen[i]} for i, t in enumerate(TICKERS)}

# 8.4 Cross-asset covariance diff (Frobenius norm)
cov_real = np.cov(real_all.T)
cov_gen  = np.cov(gen_all.T)
cov_diff = np.linalg.norm(cov_real - cov_gen, ord="fro")
report["covariance_frobenius_diff"] = float(cov_diff)

with open(OUT_DIR/"vae_validation_report.json","w") as f:
    json.dump(report, f, indent=2)

print("Validation report:", (OUT_DIR/"vae_validation_report.json").name)
print(json.dumps(report, indent=2)[:1200], " ...")


Validation report: vae_validation_report.json
{
  "ks_per_asset": {
    "AAPL": {
      "statistic": 0.46813014790468355,
      "pvalue": 0.0
    },
    "MSFT": {
      "statistic": 0.46683492193919474,
      "pvalue": 0.0
    },
    "AMZN": {
      "statistic": 0.4633198438783894,
      "pvalue": 0.0
    },
    "GOOGL": {
      "statistic": 0.48521511092851277,
      "pvalue": 0.0
    },
    "META": {
      "statistic": 0.46494448644207065,
      "pvalue": 0.0
    },
    "NVDA": {
      "statistic": 0.4674969597370583,
      "pvalue": 0.0
    },
    "JPM": {
      "statistic": 0.4752853820870994,
      "pvalue": 0.0
    },
    "JNJ": {
      "statistic": 0.47324169268693506,
      "pvalue": 0.0
    },
    "XOM": {
      "statistic": 0.47078898110106826,
      "pvalue": 0.0
    },
    "WMT": {
      "statistic": 0.4711023418241578,
      "pvalue": 0.0
    }
  },
  "moments": {
    "AAPL": {
      "real_mean": 0.0010208849910188264,
      "gen_mean": 0.0010159678058698773,
      "real_s

Hand-off summary (for Week 3)

In [10]:
# --- Cell 9: Hand-off summary ---
print("Artifacts ready for Week 3:")
for p in [
    OUT_DIR / "vae_model.pt",
    OUT_DIR / "vae_scaler.json",
    OUT_DIR / "synthetic_paths_20d.parquet",
    OUT_DIR / "synthetic_paths_20d.npy",
    OUT_DIR / "vae_validation_report.json",
]:
    print(" •", p.name)


Artifacts ready for Week 3:
 • vae_model.pt
 • vae_scaler.json
 • synthetic_paths_20d.parquet
 • synthetic_paths_20d.npy
 • vae_validation_report.json
